In [1]:
%%time
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

from binance import AsyncClient, BinanceSocketManager, Client, ThreadedWebsocketManager
import Artha.configs.binance_config as c 
key = c.apis[0][0]
secret = c.apis[0][1]
client = Client(key, secret)

import asyncio
import time
import nest_asyncio
nest_asyncio.apply()

# client.get_historical_klines("ETHBTC", "1h", "1 Dec, 2017", "1 Jan, 2018")
import pandas as pd
pd.options.display.float_format = '{:.9f}'.format
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# from Artha.mentions import *
from datetime import datetime, timedelta
import numpy as np

from Artha.trading import *

import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot

CPU times: user 1.14 s, sys: 269 ms, total: 1.41 s
Wall time: 2.16 s


In [20]:
%%time
ticker, time_frame = "ETHBTC", "4h"

df = load_klines("ETHBTC", "../data/crypto_price_data/")
ichimoku(df)

CPU times: user 170 ms, sys: 11.4 ms, total: 181 ms
Wall time: 181 ms


DatetimeIndex([          '2019-12-31 19:00:00',
               '2019-12-31 23:00:05.907449950',
               '2020-01-01 03:00:11.814899901'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [24]:
[df.bull_tk_cross.index[i] for i in range(len(df)) if df.bull_tk_cross.values[i] == 1]

[Timestamp('2020-01-14 11:08:04.410895963'),
 Timestamp('2020-02-02 03:19:06.045290449'),
 Timestamp('2020-03-06 11:38:47.535280604'),
 Timestamp('2020-03-31 11:53:33.652773220'),
 Timestamp('2020-05-19 16:22:36.350508698'),
 Timestamp('2020-06-22 04:42:23.747948802'),
 Timestamp('2020-07-04 04:49:29.084345258'),
 Timestamp('2020-07-21 08:59:37.551690188'),
 Timestamp('2020-08-13 13:13:18.687233344'),
 Timestamp('2020-08-28 21:22:22.172628816'),
 Timestamp('2020-09-12 09:30:56.120774532'),
 Timestamp('2020-09-20 01:35:27.863472268'),
 Timestamp('2020-09-28 13:40:29.143419756'),
 Timestamp('2020-10-11 09:48:04.017065968'),
 Timestamp('2020-10-25 09:56:20.242861832'),
 Timestamp('2020-11-08 22:04:54.191007548'),
 Timestamp('2020-11-21 14:12:23.157203808'),
 Timestamp('2020-11-29 10:17:00.807351492'),
 Timestamp('2020-12-04 02:19:46.215950116'),
 Timestamp('2020-12-30 10:35:19.593042336'),
 Timestamp('2021-01-03 06:37:35.464391204'),
 Timestamp('2021-01-06 06:39:21.798490320'),
 Timestamp

In [2]:
# download multiple data at once
markets = get_quote_dict()
btc_markets = [tick+asset for asset in list(markets.keys())[:1] for tick in markets[asset]]
concurrent_download_klines(btc_markets[1:15], "03/01/2021 00:00:00")

In [ ]:
# streams

In [ ]:
async def single(name):
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client, user_timeout = 20)
    # start any sockets here, i.e a trade socket
    ts = bm.kline_socket(name, interval="5m")
    # then start receiving messages
    async with ts as tscm:
        while True:
            res = await tscm.recv()
            print(res)
   
    await client.close_connection()

async def multiple (asset_names):
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client, user_timeout = 20)
    # start any sockets here, i.e a trade socket
    ms = bm.multiplex_socket(asset_names)
    # then start receiving messages
    async with ms as tscm:
        while True:
            res = await tscm.recv()
            print(res["data"]["k"])
   
    await client.close_connection()

In [ ]:
import Artha.crypto_data as crypto
markets = crypto.get_market_dict()
print("BTC" in [markets.keys()])
print("BTC" in [i for i in crypto.get_invert_dict(markets).keys()])
s_type = "@kline_5m"
btc_markets = [(tick+asset+s_type).lower() for asset in list(markets.keys())[:1] for tick in markets[asset]]

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(multiple(btc_markets[:3]))
print("done")

In [ ]:
symbol = 'BNBBTC'

twm = ThreadedWebsocketManager(api_key=key, api_secret=secret)
# start is required to initialise its internal loop
twm.start()

def handle_socket_message(msg):
    print(f"message type: {msg['e']}")
    print(msg)

twm.start_kline_socket(callback=handle_socket_message, symbol=symbol)

# multiple sockets can be started
twm.start_depth_socket(callback=handle_socket_message, symbol=symbol)

# or a multiplex socket can be started like this
# see Binance docs for stream names
streams = ['BNBBTC@miniTicker', 'BNBBTC@bookTicker']
twm.start_multiplex_socket(callback=handle_socket_message, streams=streams)

In [ ]:
# incorporated social media data
start_date, window_count = "03/01/2021 00:00:00", 63

dates = [from_datetime(to_datetime(start_date)+timedelta(days=i)) for i in range(window_count)]

pdrank = pd.read_csv("pgrankdata.csv")
pdrank['Date'] = pd.to_datetime(pdrank['Unnamed: 0'])
pdrank = pdrank.set_index("Date").drop(['Unnamed: 0'], axis = 1)

i = 0
ticker = "DOGE"

while i<len(dates)-1:
    if pdrank[ticker][i]!=0:
        di = pdrank[ticker][i+1]/pdrank[ticker][i]
        if di> 1.1:
            print(i, di, pdrank[ticker][i+1], pdrank[ticker][i])
            print(pdrank[ticker].index[i+1])
    else:
        if pdrank[ticker][i+1] > 0:
            print("new", i, pdrank[ticker][i+1])
    i+=1

In [ ]:
# direct from neo

inc = 32
wind = 30

if ticker == "BTC":
    asset = ticker+"USDT"
else:
    asset = ticker+"BTC"

interval = '2h'
newest, oldest = pdrank[ticker].index[inc-wind: inc+wind][-1].to_pydatetime().timestamp() * 1000, pdrank[ticker].index[inc-wind: inc+wind][0].to_pydatetime().timestamp() * 1000, 
df = crypto.get_klines_df(asset, interval, str(oldest), str(newest))
df.keys()


plt.rcParams['figure.figsize'] = [10, 5]
ts = df["Volume"].astype(float) #.pct_change()

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()


ax1.plot(pdrank[ticker].index[inc-wind: inc+wind], pdrank[ticker][inc-wind: inc+wind], color='b')

ax2.plot(ts.index, ts, color='g')

ax1.set_xlabel('Time')
ax1.set_ylabel(ticker+" PageRank Score", color='b')
ax2.set_ylabel(ticker +" Price", color='g')
# plt.plot([inc]*2, range(-1,1))
plt.title(asset+" "+from_datetime(pdrank[ticker].index[inc].to_pydatetime())[:-9])
plt.gcf().autofmt_xdate()